In [ ]:
## extract and clip precipiaton rasters

In [4]:
import os
import glob
import rioxarray as rxr
import geopandas as gpd

# -----------------------------
# File paths
# -----------------------------
# Already downloaded
# precip1950_59_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/precipdata/wc2.1_cruts4.09_2.5m_prec_1950-1959"
precip1960_69_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/precipdata/wc2.1_cruts4.09_2.5m_prec_1960-1969"

angola_gpkg = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_gpkg_stuff/angola_boundaries_32733.gpkg"
output_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/precip_masked"

os.makedirs(output_folder, exist_ok=True)

# -----------------------------
# Load Angola vector
# -----------------------------
angola = gpd.read_file(angola_gpkg)
print("Angola CRS:", angola.crs)

# -----------------------------
# Function to clip rasters
# -----------------------------
def clip_precip_to_angola(folder_path, angola_shape, output_folder):
    tif_files = glob.glob(os.path.join(folder_path, "*.tif"))

    # List existing files in output folder
    existing_files = set(os.listdir(output_folder))

    for tif in tif_files:
        filename = os.path.basename(tif)
        
        # Skip if already processed
        if filename in existing_files:
            print(f"Skipping {filename}, already exists.")
            continue

        print("Processing:", filename)
        precip = rxr.open_rasterio(tif, masked=True)

        # Reproject precipitation raster to Angola CRS if needed
        if precip.rio.crs != angola_shape.crs:
            precip = precip.rio.reproject(angola_shape.crs)

        # Clip to Angola polygon (vector mask)
        precip_clipped = precip.rio.clip(angola_shape.geometry, angola_shape.crs, drop=True, invert=False)
        
        # Squeeze to remove extra dimensions (single-band)
        precip_clipped = precip_clipped.squeeze()

        # Save clipped raster
        out_path = os.path.join(output_folder, filename)
        precip_clipped.rio.to_raster(out_path)
        print("Saved:", out_path)

# -----------------------------
# Run for each decade
# -----------------------------
# Already downloaded 

# clip_precip_to_angola(precip1950_59_path, angola, output_folder)
clip_precip_to_angola(precip1960_69_path, angola, output_folder)

print("All new precipitation rasters clipped to Angola!")


Angola CRS: EPSG:32733
Skipping wc2.1_cruts4.09_2.5m_prec_1960-01.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-02.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-03.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-04.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-05.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-06.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-07.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-08.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-09.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-10.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-11.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1960-12.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1961-01.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1961-02.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_prec_1961-03.tif, already exists.
Skipping wc2.1_cr

Precipitation sum per year

In [2]:
import os
import re
import glob
import rioxarray as rxr
import numpy as np
import xarray as xr 

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# Change these to your actual folders:
precip_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/precip_masked"

output_base   = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_precip_sum"

os.makedirs(output_base, exist_ok=True)

import os
import re
import glob
import rioxarray as rxr
import numpy as np
import xarray as xr   # <-- ADD THIS LINE

# ---------------------------------------------------------
# Helper function to extract year from filename
# ---------------------------------------------------------
def extract_year(fname):
    match = re.search(r"(\d{4})", fname)
    return match.group(1) if match else None

# ---------------------------------------------------------
# Generic function to compute annual aggregation
# ---------------------------------------------------------
def annual_aggregate(input_folder, output_folder, variable):
    tif_files = sorted(glob.glob(os.path.join(input_folder, "*.tif")))
    yearly_groups = {}

    # group files by year
    for tif in tif_files:
        year = extract_year(tif)
        if not year:
            continue
        yearly_groups.setdefault(year, []).append(tif)

    for year, files in yearly_groups.items():
        print(f"\nProcessing {variable} for {year} ({len(files)} monthly files)...")

        # load and stack all months for this year
        arrays = [rxr.open_rasterio(f, masked=True).squeeze() for f in files]
        stacked = xr.concat(arrays, dim="month")   # <-- FIXED HERE

        # aggregate
        if variable == "precip":
            annual = stacked.sum(dim="month")  # total annual precip
        else:
            annual = stacked.mean(dim="month")  # average annual temperature

        # save
        out_path = os.path.join(output_folder, f"{variable}_{year}_annual.tif")
        annual.rio.to_raster(out_path)
        print(f"Saved: {out_path}")


# ---------------------------------------------------------
# Run for each variable
# ---------------------------------------------------------
os.makedirs(os.path.join(output_base, "precip"), exist_ok=True)

annual_aggregate(precip_folder, os.path.join(output_base, "precip"), "precip")

print("\n✅ Annual climate rasters created for precip.")



Processing precip for 1951 (12 monthly files)...
Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_precip_sum/precip/precip_1951_annual.tif

Processing precip for 1952 (12 monthly files)...
Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_precip_sum/precip/precip_1952_annual.tif

Processing precip for 1953 (12 monthly files)...
Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_precip_sum/precip/precip_1953_annual.tif

Processing precip for 1954 (12 monthly files)...
Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_precip_sum/precip/precip_1954_annual.tif

Processing precip for 1955 (12 monthly files)...
Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_precip_sum/precip/precip_1955_annual.tif

Processing precip for 1956 (12 monthly files)...
Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_precip_sum/precip/pre

In [1]:
## tmin clipped and extracted 2.5m res

import os
import glob
import rioxarray as rxr
import geopandas as gpd

# -----------------------------
# File paths
# -----------------------------
# Already downloaded
tmin1950_59_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmindata/wc2.1_cruts4.09_2.5m_tmin_1950-1959"
tmin1960_69_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmindata/wc2.1_cruts4.09_2.5m_tmin_1960-1969"

angola_gpkg = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_gpkg_stuff/angola_boundaries_32733.gpkg"
output_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/tmin"

os.makedirs(output_folder, exist_ok=True)

# -----------------------------
# Load Angola vector
# -----------------------------
angola = gpd.read_file(angola_gpkg)
print("Angola CRS:", angola.crs)

# -----------------------------
# Function to clip rasters
# -----------------------------
def clip_tmin_to_angola(folder_path, angola_shape, output_folder):
    tif_files = glob.glob(os.path.join(folder_path, "*.tif"))

    # List existing files in output folder
    existing_files = set(os.listdir(output_folder))

    for tif in tif_files:
        filename = os.path.basename(tif)
        
        # Skip if already processed
        if filename in existing_files:
            print(f"Skipping {filename}, already exists.")
            continue

        print("Processing:", filename)
        tmin = rxr.open_rasterio(tif, masked=True)

        # Reproject precipitation raster to Angola CRS if needed
        if tmin.rio.crs != angola_shape.crs:
            tmin = tmin.rio.reproject(angola_shape.crs)

        # Clip to Angola polygon (vector mask)
        tmin_clipped = tmin.rio.clip(angola_shape.geometry, angola_shape.crs, drop=True, invert=False)
        
        # Squeeze to remove extra dimensions (single-band)
        tmin_clipped = tmin_clipped.squeeze()

        # Save clipped raster
        out_path = os.path.join(output_folder, filename)
        tmin_clipped.rio.to_raster(out_path)
        print("Saved:", out_path)

# -----------------------------
# Run for each decade§
# -----------------------------
# Already downloaded 

clip_tmin_to_angola(tmin1950_59_path, angola, output_folder)
clip_tmin_to_angola(tmin1960_69_path, angola, output_folder)

print("All new tmin rasters clipped to Angola!")


Angola CRS: EPSG:32733
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-01.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-02.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-03.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-04.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-05.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-06.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-07.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-08.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-09.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-10.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-11.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1951-12.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1952-01.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1952-02.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmin_1952-03.tif, already exists.
Skipping wc2.1_cr

## tmin months averaged by year

In [1]:
## timn months averaged by year
import os
import re
import glob
import rioxarray as rxr
import xarray as xr

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# Update these paths to your system
tmin_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/tmin"
output_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmin_mean"

os.makedirs(output_folder, exist_ok=True)

# ---------------------------------------------------------
# Helper function to extract year from filename
# ---------------------------------------------------------
def extract_year(fname):
    match = re.search(r"(\d{4})", fname)
    return match.group(1) if match else None

# ---------------------------------------------------------
# Aggregate monthly rasters into annual means
# ---------------------------------------------------------
def annual_mean_temperature(input_folder, output_folder, variable_name="tmin"):
    tif_files = sorted(glob.glob(os.path.join(input_folder, "*.tif")))
    yearly_groups = {}

    # group files by year
    for tif in tif_files:
        year = extract_year(tif)
        if not year:
            continue
        yearly_groups.setdefault(year, []).append(tif)

    # process each year
    for year, files in yearly_groups.items():
        print(f"\nProcessing {variable_name} for {year} ({len(files)} monthly files)...")

        # load all 12 monthly rasters
        arrays = [rxr.open_rasterio(f, masked=True).squeeze() for f in files]

        # make sure all rasters share same geospatial properties
        arrays = [arr.rio.write_crs(arrays[0].rio.crs, inplace=False) for arr in arrays]

        # stack along a new dimension (month)
        stacked = xr.concat(arrays, dim="month")

        # compute mean across months
        annual_mean = stacked.mean(dim="month")

        # preserve metadata
        annual_mean.rio.write_crs(arrays[0].rio.crs, inplace=True)
        annual_mean.rio.write_transform(arrays[0].rio.transform(), inplace=True)

        # save to file
        out_path = os.path.join(output_folder, f"{variable_name}_{year}_annual_mean.tif")
        annual_mean.rio.to_raster(out_path)
        print(f"✅ Saved: {out_path}")

# ---------------------------------------------------------
# RUN
# ---------------------------------------------------------
annual_mean_temperature(tmin_folder, output_folder, "tmin")

print("\n🎉 Annual Tmin rasters created successfully.")



Processing tmin for 1951 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmin_mean/tmin_1951_annual_mean.tif

Processing tmin for 1952 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmin_mean/tmin_1952_annual_mean.tif

Processing tmin for 1953 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmin_mean/tmin_1953_annual_mean.tif

Processing tmin for 1954 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmin_mean/tmin_1954_annual_mean.tif

Processing tmin for 1955 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmin_mean/tmin_1955_annual_mean.tif

Processing tmin for 1956 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmin_mean/tmin_1956_annual_mean.tif

Processin

In [1]:
## tmax clipped and extracted 2.5m res


import os
import glob
import rioxarray as rxr
import geopandas as gpd

# -----------------------------
# File paths
# -----------------------------
# Already downloaded
tmax1950_59_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmaxdata/wc2.1_cruts4.09_2.5m_tmax_1950-1959"
tmax1960_69_path = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmaxdata/wc2.1_cruts4.09_2.5m_tmax_1960-1969"

angola_gpkg = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_gpkg_stuff/angola_boundaries_32733.gpkg"
output_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/tmax"

os.makedirs(output_folder, exist_ok=True)

# -----------------------------
# Load Angola vector
# -----------------------------
angola = gpd.read_file(angola_gpkg)
print("Angola CRS:", angola.crs)

# -----------------------------
# Function to clip rasters
# -----------------------------
def clip_tmax_to_angola(folder_path, angola_shape, output_folder):
    tif_files = glob.glob(os.path.join(folder_path, "*.tif"))

    # List existing files in output folder
    existing_files = set(os.listdir(output_folder))

    for tif in tif_files:
        filename = os.path.basename(tif)
        
        # Skip if already processed
        if filename in existing_files:
            print(f"Skipping {filename}, already exists.")
            continue

        print("Processing:", filename)
        tmax = rxr.open_rasterio(tif, masked=True)

        # Reproject precipitation raster to Angola CRS if needed
        if tmax.rio.crs != angola_shape.crs:
            tmax = tmax.rio.reproject(angola_shape.crs)

        # Clip to Angola polygon (vector mask)
        tmax_clipped = tmax.rio.clip(angola_shape.geometry, angola_shape.crs, drop=True, invert=False)
        
        # Squeeze to remove extra dimensions (single-band)
        tmax_clipped = tmax_clipped.squeeze()

        # Save clipped raster
        out_path = os.path.join(output_folder, filename)
        tmax_clipped.rio.to_raster(out_path)
        print("Saved:", out_path)

# -----------------------------
# Run for each decade§
# -----------------------------
# Already downloaded 

clip_tmax_to_angola(tmax1950_59_path, angola, output_folder)
clip_tmax_to_angola(tmax1960_69_path, angola, output_folder)

print("All new tmax rasters clipped to Angola!")


Angola CRS: EPSG:32733
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-01.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-02.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-03.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-04.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-05.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-06.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-07.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-08.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-09.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-10.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-11.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1951-12.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1952-01.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1952-02.tif, already exists.
Skipping wc2.1_cruts4.09_2.5m_tmax_1952-03.tif, already exists.
Skipping wc2.1_cr

In [3]:
##tmax months averaged by year

import os
import re
import glob
import rioxarray as rxr
import xarray as xr

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
# Update these paths to your system
tmax_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/tmax"
output_folder = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean"

os.makedirs(output_folder, exist_ok=True)

# ---------------------------------------------------------
# Helper function to extract year from filename
# ---------------------------------------------------------
def extract_year(fname):
    match = re.search(r"(\d{4})", fname)
    return match.group(1) if match else None

# ---------------------------------------------------------
# Aggregate monthly rasters into annual means
# ---------------------------------------------------------
def annual_mean_temperature(input_folder, output_folder, variable_name="tmax"):
    tif_files = sorted(glob.glob(os.path.join(input_folder, "*.tif")))
    yearly_groups = {}

    # group files by year
    for tif in tif_files:
        year = extract_year(tif)
        if not year:
            continue
        yearly_groups.setdefault(year, []).append(tif)

    # process each year
    for year, files in yearly_groups.items():
        print(f"\nProcessing {variable_name} for {year} ({len(files)} monthly files)...")

        # load all 12 monthly rasters
        arrays = [rxr.open_rasterio(f, masked=True).squeeze() for f in files]

        # make sure all rasters share same geospatial properties
        arrays = [arr.rio.write_crs(arrays[0].rio.crs, inplace=False) for arr in arrays]

        # stack along a new dimension (month)
        stacked = xr.concat(arrays, dim="month")

        # compute mean across months
        annual_mean = stacked.mean(dim="month")

        # preserve metadata
        annual_mean.rio.write_crs(arrays[0].rio.crs, inplace=True)
        annual_mean.rio.write_transform(arrays[0].rio.transform(), inplace=True)

        # save to file
        out_path = os.path.join(output_folder, f"{variable_name}_{year}_annual_mean.tif")
        annual_mean.rio.to_raster(out_path)
        print(f"✅ Saved: {out_path}")

# ---------------------------------------------------------
# RUN
# ---------------------------------------------------------
annual_mean_temperature(tmax_folder, output_folder, "tmax")

print("\n🎉 Annual Tmax rasters created successfully.")



Processing tmax for 1951 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean/tmax_1951_annual_mean.tif

Processing tmax for 1952 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean/tmax_1952_annual_mean.tif

Processing tmax for 1953 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean/tmax_1953_annual_mean.tif

Processing tmax for 1954 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean/tmax_1954_annual_mean.tif

Processing tmax for 1955 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean/tmax_1955_annual_mean.tif

Processing tmax for 1956 (12 monthly files)...
✅ Saved: /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/annual_tmax_mean/tmax_1956_annual_mean.tif

Processin

In [3]:
import os
import glob
import zipfile
import rioxarray as rxr
import geopandas as gpd

# -----------------------------
# File paths
# -----------------------------
tmin1950_59_zip = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmindata/wc2.1_cruts4.09_2.5m_tmin_1950-1959.zip"
tmin1960_69_zip = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmindata/wc2.1_cruts4.09_2.5m_tmin_1960-1969.zip"

angola_gpkg = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_gpkg_stuff/angola_boundaries_32733.gpkg"
output_folder = "/Volumes/One_Touch/angola_soils_thesis/data_processed/tmin_masked"
os.makedirs(output_folder, exist_ok=True)

# Folders to extract zips
tmin1950_59_folder = "/Volumes/One_Touch/angola_soils_thesis/data_raw/tmin1950_59"
tmin1960_69_folder = "/Volumes/One_Touch/angola_soils_thesis/data_raw/tmin1960_69"
os.makedirs(tmin1950_59_folder, exist_ok=True)
os.makedirs(tmin1960_69_folder, exist_ok=True)

# -----------------------------
# Extract ZIP files
# -----------------------------
for zip_file, extract_folder in [(tmin1950_59_zip, tmin1950_59_folder),
                                 (tmin1960_69_zip, tmin1960_69_folder)]:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print(f"Extracted {zip_file} to {extract_folder}")

# -----------------------------
# Load Angola vector
# -----------------------------
angola = gpd.read_file(angola_gpkg)
print("Angola CRS:", angola.crs)

# -----------------------------
# Function to clip Tmin rasters
# -----------------------------
def tmin_clipped_to_angola(folder_path, angola_shape, output_folder):
    tif_files = glob.glob(os.path.join(folder_path, "*.tif"))

    for tif in tif_files:
        print("Processing:", os.path.basename(tif))
        tmin = rxr.open_rasterio(tif, masked=True)

        # Reproject raster to Angola CRS if needed
        if tmin.rio.crs != angola_shape.crs:
            tmin = tmin.rio.reproject(angola_shape.crs)

        # Clip to Angola polygon (vector mask)
        tmin_clipped = tmin.rio.clip(angola_shape.geometry, angola_shape.crs, drop=True, invert=False)
        
        # Squeeze to remove extra dimensions (single-band)
        tmin_clipped = tmin_clipped.squeeze()

        # Save clipped raster
        out_path = os.path.join(output_folder, os.path.basename(tif))
        tmin_clipped.rio.to_raster(out_path)
        print("Saved:", out_path)

# -----------------------------
# Run for each decade
# -----------------------------
tmin_clipped_to_angola(tmin1950_59_folder, angola, output_folder)
tmin_clipped_to_angola(tmin1960_69_folder, angola, output_folder)

print("All Tmin rasters clipped to Angola!")


Extracted /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmindata/wc2.1_cruts4.09_10m_tmin_1950-1959.zip to /Volumes/One_Touch/angola_soils_thesis/data_raw/tmin1950_59
Extracted /Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmindata/wc2.1_cruts4.09_10m_tmin_1960-1969.zip to /Volumes/One_Touch/angola_soils_thesis/data_raw/tmin1960_69
Angola CRS: EPSG:32733
Processing: wc2.1_cruts4.09_10m_tmin_1951-01.tif
Saved: /Volumes/One_Touch/angola_soils_thesis/data_processed/tmin_masked/wc2.1_cruts4.09_10m_tmin_1951-01.tif
Processing: wc2.1_cruts4.09_10m_tmin_1951-02.tif
Saved: /Volumes/One_Touch/angola_soils_thesis/data_processed/tmin_masked/wc2.1_cruts4.09_10m_tmin_1951-02.tif
Processing: wc2.1_cruts4.09_10m_tmin_1951-03.tif
Saved: /Volumes/One_Touch/angola_soils_thesis/data_processed/tmin_masked/wc2.1_cruts4.09_10m_tmin_1951-03.tif
Processing: wc2.1_cruts4.09_10m_tmin_1951-04.tif
Saved: /Volumes/One_Touch/angola_soils_thesis/data_processed/tmin

In [ ]:
## need to extract tmean!

import os
import glob
import zipfile
import rioxarray as rxr
import geopandas as gpd

# -----------------------------
# File paths
# -----------------------------
tmin1950_59_zip = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmindata/wc2.1_cruts4.09_10m_tmin_1950-1959.zip"
tmin1960_69_zip = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_raw/climaticdata/tmindata/wc2.1_cruts4.09_10m_tmin_1960-1969.zip"

angola_gpkg = "/Volumes/One_Touch/angola_soils_thesis/GIS_Angola/data_processed/angola_soil_gpkg_stuff/angola_boundaries_32733.gpkg"
output_folder = "/Volumes/One_Touch/angola_soils_thesis/data_processed/tmin_masked"
os.makedirs(output_folder, exist_ok=True)

# Folders to extract zips
tmin1950_59_folder = "/Volumes/One_Touch/angola_soils_thesis/data_raw/tmin1950_59"
tmin1960_69_folder = "/Volumes/One_Touch/angola_soils_thesis/data_raw/tmin1960_69"
os.makedirs(tmin1950_59_folder, exist_ok=True)
os.makedirs(tmin1960_69_folder, exist_ok=True)

# -----------------------------
# Extract ZIP files
# -----------------------------
for zip_file, extract_folder in [(tmin1950_59_zip, tmin1950_59_folder),
                                 (tmin1960_69_zip, tmin1960_69_folder)]:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print(f"Extracted {zip_file} to {extract_folder}")

# -----------------------------
# Load Angola vector
# -----------------------------
angola = gpd.read_file(angola_gpkg)
print("Angola CRS:", angola.crs)

# -----------------------------
# Function to clip Tmin rasters
# -----------------------------
def tmin_clipped_to_angola(folder_path, angola_shape, output_folder):
    tif_files = glob.glob(os.path.join(folder_path, "*.tif"))

    for tif in tif_files:
        print("Processing:", os.path.basename(tif))
        tmin = rxr.open_rasterio(tif, masked=True)

        # Reproject raster to Angola CRS if needed
        if tmin.rio.crs != angola_shape.crs:
            tmin = tmin.rio.reproject(angola_shape.crs)

        # Clip to Angola polygon (vector mask)
        tmin_clipped = tmin.rio.clip(angola_shape.geometry, angola_shape.crs, drop=True, invert=False)
        
        # Squeeze to remove extra dimensions (single-band)
        tmin_clipped = tmin_clipped.squeeze()

        # Save clipped raster
        out_path = os.path.join(output_folder, os.path.basename(tif))
        tmin_clipped.rio.to_raster(out_path)
        print("Saved:", out_path)

# -----------------------------
# Run for each decade
# -----------------------------
tmin_clipped_to_angola(tmin1950_59_folder, angola, output_folder)
tmin_clipped_to_angola(tmin1960_69_folder, angola, output_folder)

print("All Tmin rasters clipped to Angola!")
